In [1]:
pip install -U finance_datareader


In [2]:
pip install xmltodict

In [3]:
import pandas as pd
import FinanceDataReader as fdr
from time import time
from concurrent.futures import ProcessPoolExecutor

In [4]:
#from google.colab import drive
#drive.mount('/content/drive')
#from google.colab import files
#myfile = files.upload()

In [5]:
#filename = 'data.csv'
#new_stock_code = pd.read_csv(filename)
#new_stock_code.head()

In [6]:
df_ksp = fdr.StockListing('KOSPI')
df_ksp.head()


,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,이현우,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
6,027410,KOSPI,BGF,기타 금융업,지주회사,2014-05-19,12월,홍정국,http://www.bgf.co.kr,서울특별시
7,282330,KOSPI,BGF리테일,종합 소매업,체인화 편의점,2017-12-08,12월,이건준,http://www.bgfretail.com,서울특별시
8,138930,KOSPI,BNK금융지주,기타 금융업,금융지주회사,2011-03-30,12월,김지완,http://www.bnkfg.com,부산광역시


In [7]:
stock_code = df_ksp[['Symbol','Name']]
#stock_code.to_excel('code.xlsx')


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import xmltodict
import json

In [15]:
def get_daily_stock_price(stockCode,name,count):
  global df
  if (len(stockCode) == 6 ) :
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    url=f'https://fchart.stock.naver.com/sise.nhn?symbol={stockCode}&timeframe=day&count={count}&requestType=0'
    #print(url)
    rs = requests.get(url,headers = headers)
    dt = xmltodict.parse(rs.text)
    js = json.dumps(dt,indent=4)
    js = json.loads(js)
    data = pd.json_normalize(js['protocol']['chartdata']['item'])
    df = data['@data'].str.split('|',expand=True)
    df.columns = ['data','open','high','low','close','Volume']
    df['name'] = str(name)  ## NAME 행을 만들어 기업명 추가
  return df

In [16]:
tmp=[]

for index,row in stock_code.iterrows():
  tmp.append(get_daily_stock_price(row['Symbol'],row['Name'],'2500'))
  #tmp.append(get_daily_stock_price(row['종목코드'],row['기업명'],'2'))
  #print(tmp)
  #tmp.append(get_daily_stock_price(row['종목코드']))

In [18]:
tmp

[          data  open  high   low close   Volume    name
 0     20150821  6180  7300  5600  7100  1994022  AJ네트웍스
 1     20150824  6700  6940  6240  6480   392831  AJ네트웍스
 2     20150825  6610  6730  6189  6220   242730  AJ네트웍스
 3     20150826  6260  7760  6260  7090   703721  AJ네트웍스
 4     20150827  7239  8060  6960  7570   499913  AJ네트웍스
 ...        ...   ...   ...   ...   ...      ...     ...
 1296  20201201  4560  4560  4445  4460   208335  AJ네트웍스
 1297  20201202  4460  4505  4350  4365   243752  AJ네트웍스
 1298  20201203  4450  4535  4355  4480   198469  AJ네트웍스
 1299  20201204  4495  4575  4495  4550   222605  AJ네트웍스
 1300  20201207  4510  4550  4455  4455   155605  AJ네트웍스
 
 [1301 rows x 7 columns],
           data   open   high    low  close  Volume   name
 0     20101019  21005  22038  20787  21632   25949  AK홀딩스
 1     20101020  21631  22289  21100  21507   26497  AK홀딩스
 2     20101021  21569  21913  21224  21226   23380  AK홀딩스
 3     20101022  21287  22477  21225  22478   49003 

In [19]:
df_final = pd.concat(tmp,ignore_index=True)

In [20]:
df_final

,data,open,high,low,close,Volume,name
0,20150821,6180,7300,5600,7100,1994022,AJ네트웍스
1,20150824,6700,6940,6240,6480,392831,AJ네트웍스
2,20150825,6610,6730,6189,6220,242730,AJ네트웍스
3,20150826,6260,7760,6260,7090,703721,AJ네트웍스
4,20150827,7239,8060,6960,7570,499913,AJ네트웍스
...,...,...,...,...,...,...,...
2236123,20201201,0,0,0,258,0,흥아해운
2236124,20201202,0,0,0,258,0,흥아해운
2236125,20201203,0,0,0,258,0,흥아해운
2236126,20201204,0,0,0,258,0,흥아해운


In [21]:

df_final.to_csv('NEW_KOSPI_10_years_data.csv',encoding='euc-kr')
